In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine Learning/NLP/News Classification/fake_or_real_news.csv")
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [4]:
df.drop(["Unnamed: 0", "title"], axis = 1, inplace = True)

In [5]:
df["label"].value_counts()

,count
label,
REAL,3171
FAKE,3164


# **Model**

In [6]:
import re

import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
sw = stopwords.words("english")

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def mystemmer(x):
    return " ".join([ps.stem(word) for word in re.sub("[^a-zA-Z]", " ", x).lower().split() if not word in sw])

# def mystemmer(x):
#     temp = re.sub("[^a-zA-Z]", " ", x)
#     temp = temp.lower()
#     temp = temp.split()
#     temp = [ps.stem(word) for word in temp if word not in sw]
#     temp = " ".join(temp)
#     return temp

In [8]:
df["text"] = df["text"].apply(mystemmer)

In [9]:
df

,text,label
0,daniel greenfield shillman journal fellow free...,FAKE
1,googl pinterest digg linkedin reddit stumbleup...,FAKE
2,u secretari state john f kerri said monday sto...,REAL
3,kayde king kaydeek novemb lesson tonight dem l...,FAKE
4,primari day new york front runner hillari clin...,REAL
...,...,...
6330,state depart told republican nation committe c...,REAL
6331,p pb stand plutocrat pentagon post oct wikimed...,FAKE
6332,anti trump protest tool oligarchi reform alway...,FAKE
6333,addi ababa ethiopia presid obama conven meet l...,REAL


In [10]:
x = df["text"].values
y = df["label"].values

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(x)

**What is 'stratify' ?**

* In train-test split (TTS), the stratify parameter ensures that the training and testing datasets have the same proportion of each class as in the original dataset.
* It is typically used when dealing with classification tasks to maintain the balance of classes across splits.

In [12]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x, y, test_size = 0.3, stratify = y, random_state = 2)

In [13]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [14]:
y_pred = lr.predict(x_test)

In [15]:
from sklearn.metrics import accuracy_score as acs
acs(y_test, y_pred)

0.9221462388216728

## **Random test**

In [16]:
news = "Hillary Clinton told a Staten Island crowd today that she was the candidate who could reach across party lines to get things done as president—pointing to her experience representing the borough in the Senate and even giving public thanks to Republican President George W. Bush."

In [17]:
import numpy as np
news = tfidf.transform(np.array([mystemmer(news)]))
lr.predict(news)

array(['REAL'], dtype=object)